In [1]:
import requests
import pandas as pd
import json
from bs4 import BeautifulSoup
import config
import datetime
from datetime import timedelta
import time
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

In [2]:
#mydate = datetime.date(2018,10,2)
#fecha = mydate.strftime("%Y-%m-%d")

In [3]:
#df = pd.read_csv('./clarin_full_notas.csv', index_col=0)

In [4]:
path_to_extension = r'C:\Users\p_bag\OneDrive\Documentos\1.17.0_0'
windows = '/Users/p_bag/Downloads/chromedriver_win32/chromedriver.exe'
chrome_options = webdriver.ChromeOptions()
#chrome_options.add_argument("--incognito")
chrome_options.add_argument('load-extension=' + path_to_extension)
chrome_options.add_argument("--start-maximized")

In [12]:
def scrapear(urls):
    comentarios= []
    for nota_id, url in zip(urls.index, urls.values):
        driver = webdriver.Chrome(windows, chrome_options=chrome_options)
        driver.create_options()
        driver.get(url)

        #Creo que en Clarin no es necesario scrollear para que aparezca el boton
        #driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        try:
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'pull-left')))
            print ("Hay boton")
            button = True
        except TimeoutException:
            print ("No hay boton")
            button = False  
        
        while button:
            try:
                #print('boton')
                element = driver.find_element_by_class_name('content_numb_comment')
                element.location_once_scrolled_into_view
                #time.sleep(2)
                driver.find_element_by_class_name("content_numb_comment").click()
                WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.CLASS_NAME, 'gig-comments-more')))
                button = False
            except: 
                button = False

        button = True
        while button:
            try:
                element = driver.find_element_by_class_name("gig-comments-more")
                element.location_once_scrolled_into_view
                driver.find_element_by_class_name("gig-comments-more").click()
                time.sleep(0.5)
                WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.CLASS_NAME, 'gig-comments-more')))

            except:
                button = False   

            try:
                todo = driver.find_element_by_class_name("gig-comments-comments")
                a= todo.find_elements_by_class_name("gig-comment")
                for item in a:
                    try:
                        comentarios.append({'texto': item.find_elements_by_class_name("gig-comment-body")[0].text,
                                            'usuario' : item.find_elements_by_class_name("gig-comment-username")[0].text,
                                            'likes' : item.find_elements_by_class_name("gig-comment-vote-pos")[0].text,
                                            'unLikes' :item.find_elements_by_class_name("gig-comment-vote-neg")[0].text,
                                            'fecha' : item.find_elements_by_class_name("gig-comment-time")[0].text,
                                            'respuestas' : len(item.find_elements_by_class_name('gig-comment-replies')) -1,
                                            'es_respuesta': 'gig-comment-replies' in item.find_element_by_xpath('..').get_attribute('class'),
                                            'nota_id': nota_id,
                                            'nota_url': url
                                            })
                        print('.', end = '')
                    except:
                        pass
            except:
                comentarios.append({'texto': "NOTA SIN COMENTARIOS",
                                            'usuario' : "",
                                            'likes' : "",
                                            'unlikes' "",
                                            'fecha' : "",
                                            'respuestas' : "",
                                            'es_respuesta': "",
                                            'nota_id': nota_id,
                                            'nota_url': url
                                           })
            driver.quit()
    return comentarios

In [ ]:
corte = df.url.shape[0]//4
lista_urls = []
lista_urls.append(df.url[:corte])
lista_urls.append(df.url[corte:corte*2])
lista_urls.append(df.url[corte*2:corte*3])
lista_urls.append(df.url[corte*3:df.shape[0]])

In [ ]:
%%time
from joblib import Parallel, delayed
results = Parallel(n_jobs=-1)(delayed(scrapear)(urls) for urls in lista_urls)

In [ ]:
comentarios = results[0] + results[1] + results[2] + results[3]

In [ ]:
import pandas as pd
df_comentarios = pd.DataFrame(comentarios)
df_comentarios

In [85]:
#aca dejo los elementos sueltos

driver = webdriver.Chrome(windows,chrome_options=chrome_options)
# for nota_id, url in zip(urls.index, urls.values):
url = 'https://www.clarin.com/sociedad/incidentes-puerta-legislatura-portena-gases-lacrimogenos-unicaba_0_PbVycsb-3.html'
nota_id = '1'
#url = df.url[0]
driver.get(url)

In [86]:
try:
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'pull-left')))
    print ("Hay boton")
    button = True
except TimeoutException:
    print ("No hay boton")
    button = False

Hay boton


In [87]:
while button:
    try:
        #print('boton')
        element = driver.find_element_by_class_name('content_numb_comment')
        element.location_once_scrolled_into_view
        #time.sleep(2)
        driver.find_element_by_class_name("content_numb_comment").click()
        WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.CLASS_NAME, 'gig-comments-more')))
        button = False

    except: 
        button = False

In [88]:
button = True
while button:
    try:
        element = driver.find_element_by_class_name("gig-comments-more")
        element.location_once_scrolled_into_view
        driver.find_element_by_class_name("gig-comments-more").click()
        time.sleep(0.5)
        WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.CLASS_NAME, 'gig-comments-more')))

    except:
        button = False

In [105]:
comentarios = []
todo = driver.find_element_by_class_name("gig-comments-comments")
a= todo.find_elements_by_class_name("gig-comment")
for item in a:
    comentarios.append({'texto': item.find_elements_by_class_name("gig-comment-body")[0].text,
    'usuario' : item.find_elements_by_class_name("gig-comment-username")[0].text,
    'likes' : item.find_elements_by_class_name("gig-comment-vote-pos")[0].text,
    'unLikes' :item.find_elements_by_class_name("gig-comment-vote-neg")[0].text,
    'fecha' : item.find_elements_by_class_name("gig-comment-time")[0].text,
    'respuestas' : len(item.find_elements_by_class_name('gig-comment-replies')) -1,
    'es_respuesta': 'gig-comment-replies' in item.find_element_by_xpath('..').get_attribute('class'),
    'nota_id': nota_id,
    'nota_url': url
    })